In [337]:
capital = 700  # начальный капитал
step = 100  # шаг между управлениями
profit_of_investment = [[0, 28, 45, 65, 78, 90, 102, 113],  # матрица с прибылью от вложений
                        [0, 25, 41, 55, 65, 75, 80, 85],
                        [0, 15, 25, 40, 50, 62, 73, 82],
                        [0, 20, 33, 42, 48, 53, 56, 58]]
# profit_of_investment = [[100, 100, 100, 100, 100, 100], # все работает, даже если матрица не квадратная
#                         [0, 25, 41, 41, 65],
#                         [0, 15, 25, 40, 50, 62, 73, 82],
#                         [42, 42]]

In [340]:
def calc_possible_u(profit_of_investment, step):
    """
    Считаем возможные управления, т.е. такие управления, которые вообще можно делать на k-ом шаге. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :return: лист из листов возможных управлений
    """
    U_matrix = []
    for k in range(0, len(profit_of_investment)):  # идем по предприятиям
        U = [i * step for i in range(len(profit_of_investment[k]))]  # заполняем строку матрицы
        U_matrix.append(U)  # добавляем строку в матрицу
    return U_matrix


def calc_possible_x(profit_of_investment, step, x_initial):
    """
    Считаем возможные состояния, т.е. такие состояния, которые вообще возможны на k-ом шаге. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :param x_initial: начальный капитал
    :return:
    """
    U_matrix = calc_possible_u(profit_of_investment, step)  # вычисляем матрицу возможных управлений
    X_matrix = []
    X_matrix.append(
        [x_initial])  # на первом шаге у нас всегда начальный рочно одно возможное состояние — начальный капитал
    for k in range(1, len(profit_of_investment)):  # идем по предприятиям
        # посчитаем все возможные состояния после того, как подействуем управлением u на k шаге на состояние x после k-1 шага
        X = []
        for x in X_matrix[k - 1]:
            for u in U_matrix[k - 1]:
                X.append(x - u)
        X = list(set(X))  # удалим дубликаты
        X = sorted(X)  # отсортируем
        X = [x for x in X if x >= 0]  # уберем отрицательные значения
        X_matrix.append(X)
    return X_matrix


def print_for_find_maximum_profit(profit, paths):
    """
    Функция для вывода максимально возможного дохода и последовательности управлений, которые к нему привели.
    :param profit: максимально возможный доход
    :param paths: последовательность управлений, которая привела к максимально возможному доходу
    """
    for i in range(1, len(paths) + 1):
        if (len(paths) > 1):
            print("Variant {0}:".format(i))
        u = paths[i - 1]
        for k in range(0, len(u)):
            print("It should be spent {0} on company {1}".format(u[k], k + 1))
        if (len(paths) > 1):
            print()
    if (len(paths) == 1):
        print()
    print("Total profit is {0}".format(profit))


def find_maximum_profit(profit_matrix, capital):
    """
    Функция для нахождение максимально возможного дохода.
    :param profit_matrix: лист из листов с доходом от предприятий
    :param capital: начальный капитал
    :return: словарь, в котором хранится максимально возможная прибыль и массив из последовательнойтей управлений, которые приведут к максимальной прибыли.
    """

    # заполняем переменные, которые понядобятся нам для вычисления
    U_matrix = calc_possible_u(profit_matrix, step)  # возможные управления
    X_matrix = calc_possible_x(profit_matrix, step, capital)  # возможные состояния
    num_companies = len(profit_matrix)  # кол-во компаний
    F = {}  # хеш-таблица, в которой мы храним все значения функции Беллмана

    for k in range(num_companies - 1, -1, -1):  # идем от (num_companies - 1) до 0

        X = [i for i in X_matrix[k]]  # заполняем лист возможных x (состояний)
        U = [i for i in U_matrix[k]]  # заполняем лист возможных u (управлений)

        for x in X:  # идем по возможным состояниям

            # посчитаем все возможные прибыли
            w_list = []  # лист из возможных доходов
            for u in U:  # перебираем все возможные управления
                if u > x:  # если наше управление предполагает траты большие, чем у нас есть денег
                    break  # то значит такое управление невалидно => выходим из цикла
                if k == num_companies - 1:  # если мы на шаге, соответсвующему последней компании
                    w_list.append(
                        [profit_matrix[k][U.index(u)], u])  # сохраняем и значение фукнции Беллмана, и управление
                else:
                    w_list.append([profit_matrix[k][U.index(u)] + F[(k + 1, x - u)]["w"],
                                   u])  # сохраняем и значение функции Беллмана, и управление

            # ищем максимальную прибыль
            w_max = 0  # максимальная прибыль
            for w, u in w_list:
                if w > w_max:
                    w_max = w
            u_max_list = []  # управления, при которых у нас достигается максимальная прибыль
            for w, u in w_list:  # сохраняем все возможные управления, соответствующие максимальное прибыли
                if w == w_max:
                    u_max_list.append(u)

            # добавляем значения функции Беллмана
            F[(k, x)] = {
                "w": w_max,
                "u": u_max_list
            }

    paths = [{  # лист из возможных управлений, которые приведут к максимальной прибыли
        "x": capital,
        "u": []
    }]
    for k in range(0, num_companies):  # обход графа в ширину
        paths_new = []
        for path in paths:
            x_of_path = path["x"]
            u_of_path = path["u"]
            U = F[(k, x_of_path)]["u"]
            for u in U:
                paths_new.append({
                    "x": x_of_path - u,
                    "u": [*u_of_path, u]
                })
        paths = paths_new
    u_paths = [path["u"] for path in paths]

    not_invested = capital - sum(u_paths[0])  # остаток, который мы не потратили

    maximum_profit = {
        "profit": F[(0, capital)]["w"] + not_invested,
        "paths": u_paths,
        "bellman_steps": F
    }

    print_for_find_maximum_profit(maximum_profit["profit"], maximum_profit["paths"])

    return maximum_profit

In [341]:
maximum_profit = find_maximum_profit(profit_of_investment, capital)

It should be spent 300 on company 1
It should be spent 200 on company 2
It should be spent 100 on company 3
It should be spent 100 on company 4

Total profit is 141
